In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json

with open('/content/drive/MyDrive/Shopee/contacts.json', 'r') as f:
    dict_contact = json.load(f)

In [3]:
import pandas as pd

df = pd.DataFrame(dict_contact)

In [4]:
import numpy as np

df = df.replace("", np.nan, regex=True)

In [5]:
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN


In [6]:
df_email = df.groupby(['Email']).aggregate(lambda x: list(x))
df_phone = df.groupby(['Phone']).aggregate(lambda x: list(x))
df_orderid = df.groupby(['OrderId']).aggregate(lambda x: list(x))

In [7]:
l = df_email['Id'].to_list() + df_phone['Id'].to_list() + df_orderid['Id'].to_list()

In [8]:
import networkx 
from networkx.algorithms.components.connected import connected_components

def to_graph(l):
    G = networkx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

In [9]:
G = to_graph(l)
collect = []
for i in connected_components(G):
  j = sorted(i)
  collect.append(list(j))

In [10]:
dict_final = {}
for id_trace in collect:
  for id in id_trace:
    dict_final[id] = list(id_trace)

df_final = pd.DataFrame([(i, j) for i, j in dict_final.items()], 
                   columns=['keys','values'])
df_final = df_final.sort_values('keys')
df_final = df_final.reset_index()
df_final = df_final.drop(columns=['index'])

In [11]:
df['trace_id'] = df_final['values']

In [12]:
kontak = []
for i in df['trace_id']:
  sumasi = sum([df['Contacts'][j] for j in i])
  kontak.append(sumasi)
df['sum'] = kontak

In [13]:
df.head()

,Id,Email,Phone,Contacts,OrderId,trace_id,sum
0,0,gkzAbIy@qq.com,NaN,1,NaN,[0],1
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"[1, 2458, 98519, 115061, 140081, 165605, 476346]",12
2,2,NaN,9125983679,0,NaN,"[2, 159312, 322639, 348955]",4
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,[3],0
4,4,NaN,300364407,2,NaN,[4],2


In [14]:
joint_trace_id = []
for i in df['trace_id']:
  string_trace = [str(j) for j in i]
  joint_trace_id.append("-".join(string_trace))

In [16]:
answer = []
for i in range(500000):
   trace_contacts = joint_trace_id[i] + ', ' + str(df['sum'][i])
   answer.append(trace_contacts)
df['ticket_trace/contact'] = answer

In [17]:
df_submission = df[["Id", "ticket_trace/contact"]]

In [19]:
df_submission.head()

,Id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"


In [20]:
df_submission.to_csv('/content/submission.csv', index=False)